This is going to be the workspace for the SimPy tutorial.  This is the first example. 

In [1]:
import simpy

In [42]:
env = simpy.Environment()
car = Car(env)

In [43]:
class Car(object):
    def __init__(self, env):
        self.env = env
        #start the run process every time 
        self.action = env.process(self.run())
        
    def run(self):
        while True:
            print('Start parking and charging at %d' % self.env.now)
            charge_duration = 5
            try:
                yield self.env.process(self.charge(charge_duration))
            except simpy.Interrupt:
                print('was interrupted. Hope the battery is full enough!')

            print('Start driving at %d' % self.env.now)
            trip_duration = 2
            yield self.env.timeout(trip_duration)
        
    def charge(self, duration):
        yield self.env.timeout(duration)
        


Run the first example tbh, I edited the code above so if this is rerun, we'll get the seocond example. 

In [44]:
env.run(until=15)

Start parking and charging at 0
Start driving at 5
Start parking and charging at 7
Start driving at 12
Start parking and charging at 14


The second example demonstrates interrupting an action.  Now we have a driver interrupting the car's actions. 

In [45]:
def driver(env, car):
    yield env.timeout(3)
    car.action.interrupt()

In [50]:
env = simpy.Environment()
car = Car(env) 
env.process(driver(env, car))

<Process(driver) object at 0x1057b6680>

In [51]:
env.run(until=15)

Start parking and charging at 0
was interrupted. Hope the battery is full enough!
Start driving at 3
Start parking and charging at 5
Start driving at 10
Start parking and charging at 12


Next we'll modify the scenario again to include a battery charging station which has only two spots - a limited resource.  

In [64]:
def car(env, name, bcs, driving_time, charge_duration):
    # simulate driving to the BCS
    yield env.timeout(driving_time)
    
    # request oone of the charging spots
    print('%s arriving at %d' % (name, env.now))
    with bcs.request() as req:
        # forgetting these ()'s caused AttributeError: __enter__
        yield req
        
        #charge the battery
        print ('%s starting to charge at %s' % (name, env.now))
        yield env.timeout(charge_duration)
        print('%s leaving the bcs at %s' % (name, env.now))

In [65]:
#reinit all this
import simpy
env = simpy.Environment()
bcs = simpy.Resource(env, capacity =2) 

In [66]:
for i in range(4): 
    env.process(car(env,'Car %d' % i, bcs, i*2, 5))

In [67]:
env.run()

Car 0 arriving at 0
Car 0 starting to charge at 0
Car 1 arriving at 2
Car 1 starting to charge at 2
Car 2 arriving at 4
Car 0 leaving the bcs at 5
Car 2 starting to charge at 5
Car 3 arriving at 6
Car 1 leaving the bcs at 7
Car 3 starting to charge at 7
Car 2 leaving the bcs at 10
Car 3 leaving the bcs at 12
